# Data Engineering

This notebook performs data engineering and exports the results.

In [1]:
from typing import List
from typing import Dict
from typing import Any
from typing import Optional
from typing import Set
from typing import Tuple

import os
import copy
from collections import OrderedDict
from collections import defaultdict
import pickle

import joblib
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt

%matplotlib inline
np.random.seed(42)

In [2]:
data_static: pd.DataFrame = pd.read_csv('raw_data/Static.csv', sep=';').drop('CLIENT_ID', axis=1)
data_repay: pd.DataFrame  = pd.read_csv('raw_data/Repayments.csv', sep=';')

## DataTable API
The cell below implements DataTable and API to feel it.
The entry point is described in the function `make_source_datatable`, which takes `data_static` and `data_repay` as its arguments.

In [3]:
# API for DataTable and feature engineering.

class DataTable:
    """ Stores DataFrame with engineered features, as well as 
    a structure over column names (which represent engineered features).
    
    Attributes:
        matrix -- DataFrame with features.
        
        id_payment_features -- List with non-transformed payment-realated features names in original scale.
        log_payment_features -- List with log-transformed payment-related features names in original scale.
        id_relative_payment_features -- List with non-transformed payment-related features names in relative scale.
        log_relative_payment_featurs -- List with log-transformed payment-related features names in relative scale.
        
        id_target_exact_key -- List with non-transformed target column key (next repayment) in original scale.
        log_target_exact_key -- List with log-transformed target column key (next repayment) in original scale.
        id_target_relative_key -- List with non-transformed target column key (next repayment) in relative scale.
        log_target_relative_key -- List with log-transformed target column key (next repayment) in relative scale.
        
        explicit_target_keys -- List with all keys that represent target column.
        indicator_features -- List with 0-1 indicator features names.
        id_numeric_features -- List with non-transformed numeric features names.
        log_numeric_features -- List with log-transformed numeric features names.
        
    """
    def __init__(self) -> None:
        self.matrix: Optional[pd.DataFrame] = None
        
        self.log_payment_features: List[str] = list()
        self.id_payment_features: List[str]   = ['REPAYMENT_SCHEDULED']
        self.id_relative_payment_features: List[str]  = list()
        self.log_relative_payment_features: List[str] = list()
        
        self.id_target_exact_key: str  = 'REPAYMENT_ACTUAL'
        self.log_target_exact_key: str = 'LOG_REPAYMENT_ACTUAL'
        self.id_target_relative_key: str  = 'PERCENT_ACTUAL'
        self.log_target_relative_key: str = 'LOG_PERCENT_ACTUAL'
        
        self.explicit_target_keys: List[str] = [ 
            self.id_target_exact_key, self.log_target_exact_key, 
            self.id_target_relative_key, self.log_target_relative_key
        ]

        self.indicator_features: List[str] = ['GENDER']
        self.id_numeric_features: List[str]  = [
            'PERIOD_ID', 'TERM', 'CONTRACT_SUM', 'AGE', 
            'LOAN_TO_INCOME', 'PAYMENT_TO_INCOME', 'DOWNPAYMENT', 
            'CAR_CATEGORY', 'GRACE_PERIOD', 'RATE_CHANGE_AFTER_GRACE'
        ]
        self.log_numeric_features: List[str] = list()
        

def _make_payment_cumulative_and_average_features(data_table: DataTable) -> None:
    """ Adds non-transformed cumulative and average 
    payment features to the DataTable. 
    
    Arguments:
        data_table: Structure storing DataFrame with engineered features
            alongside with convenient column names.
    """
    data: pd.DataFrame = data_table.matrix
    group_structure: pd.GroupBy = data.groupby('CONTRACT_ID')
    
    cumulative_sum_scheduled: List[float] = list()
    cumulative_sum_actual: List[float] = list()
    average_repayment_actual: List[float] = list()
    
    key: str; df: pd.DataFrame
    for key, df in group_structure:
        cumulative_sum_scheduled += df['REPAYMENT_SCHEDULED'].cumsum().tolist()
        cumulative_sum_actual += (df['REPAYMENT_ACTUAL'].cumsum() - df['REPAYMENT_ACTUAL']).tolist()
        
        cumsum_actual: Union[np.array, List[float]] = np.array(df['REPAYMENT_ACTUAL'].cumsum().tolist())
        counts_actual: Union[np.array, List[int]] = np.array(range(1, len(cumsum_actual) + 1))
        average_actual: Union[np.array, List[float]] = cumsum_actual / counts_actual
        corrected_average_history: List[float] = [0.]
        
        item: float
        for item in average_actual.tolist():
            corrected_average_history.append(item)
        corrected_average_history.pop()
        average_repayment_actual += corrected_average_history
        
    data['CUMSUM_REPAYMENT_SCHEDULED'] = cumulative_sum_scheduled
    data['CUMSUM_REPAYMENT_ACTUAL'] = cumulative_sum_actual
    data['AVERAGE_REPAYMENT_ACTUAL'] = average_repayment_actual
    data_table.id_payment_features += [
        'CUMSUM_REPAYMENT_SCHEDULED', 'CUMSUM_REPAYMENT_ACTUAL', 'AVERAGE_REPAYMENT_ACTUAL']
        

def _make_payment_history_features(data_table: DataTable) -> None:
    """ Adds payment history features to the DataTable. 
    
    Arguments:
        data_table: Structure storing DataFrame with engineered features
            alongside with convenient column names.
    """
    
    data: pd.DataFrame = data_table.matrix
    group_structure: pd.GroupBy = data.groupby('CONTRACT_ID')
    
    lagged_actual_payments: Dict[int, List[float]] = defaultdict(list)
    lagged_scheduled_payments: Dict[int, List[float]] = defaultdict(list)
    
    configurations: List[Tuple[List[float], int, str]] = [
        (lagged_actual_payments[1], 1, 'REPAYMENT_ACTUAL'),
        (lagged_scheduled_payments[1], 1, 'REPAYMENT_SCHEDULED'),
        (lagged_actual_payments[2], 2, 'REPAYMENT_ACTUAL'),
        (lagged_scheduled_payments[2], 2, 'REPAYMENT_SCHEDULED')
    ]
                         
    key: str; df: pd.DataFrame
    list_holder: List[float]; shift: int; key: str
    for key, df in group_structure:
        for list_holder, shift, key in configurations:
            contract_list: List[float] = df[key].shift(shift).tolist()
            idx: int
            for idx in range(shift):
                contract_list[idx] = df['REPAYMENT_SCHEDULED'].values[idx]
            list_holder += contract_list
    
    for list_holder, shift, key in configurations:
        new_key: str = 'HISTORY_SHIFT_{}_'.format(shift) + key
        data[new_key] = list_holder
        data_table.id_payment_features.append(new_key)
        

def _make_payment_relative_features(data_table: DataTable) -> None:
    """ Adds relative payment features to the DataTable. 
    
    Arguments:
        data_table: Structure storing DataFrame with engineered features
            alongside with convenient column names.
    """
    matrix: pd.DataFrame = data_table.matrix
    exact_features: List[str] = copy.deepcopy(data_table.id_payment_features)
    
    item: str
    for item in exact_features:
        relative_key: str = 'RELATIVE_' + item
        matrix['RELATIVE_' + item] = matrix[item] / matrix['CONTRACT_SUM'] * 100
        data_table.id_relative_payment_features.append(relative_key)

        
def _make_payment_log_features(data_table: DataTable) -> None:
    """ Adds log-transformed payment features to the DataTable. 
    
    Arguments:
        data_table: Structure storing DataFrame with engineered features
            alongside with convenient column names.
    """
    matrix: pd.DataFrame = data_table.matrix
    configurations: List[Tuple[List[str], List[str]]] = [
        (data_table.id_payment_features, data_table.log_payment_features),
        (data_table.id_relative_payment_features, data_table.log_relative_payment_features)
    ]
    source: List[str]; target: List[str]
    for source, target in configurations:
        feature_key: str
        for feature_key in source:
            log_key: str = 'LOG_' + feature_key
            matrix[log_key] = np.log1p(matrix[feature_key])
            target.append(log_key)
    

def _make_regular_log_features(data_table: DataTable) -> None:
    """ Adds regular log-transformed features to the DataTable. 
    
    Arguments:
        data_table: Structure storing DataFrame with engineered features
            alongside with convenient column names.
    """
    matrix: pd.DataFrame = data_table.matrix
    feature_key: str
    for feature_key in data_table.id_numeric_features:
        log_key: str = 'LOG_' + feature_key
        matrix[log_key] = np.log1p(matrix[feature_key])
        data_table.log_numeric_features.append(log_key)
        
        
def _make_custom_features(data_table: DataTable) -> None:
    """ Adds custom features to the DataTable. 
    This includes indicators if the grace is on, the ratio thresholds
    of loan size to income, number of period and type of car. 
    Also, "before grace" feature is added, which describes payment
    to load ratio. This way it allows to distinguish grace-free and
    regular periods.
    
    Arguments:
        data_table: Structure storing DataFrame with engineered features
            alongside with convenient column names.
    """
    matrix: pd.DataFrame = data_table.matrix
    matrix['BEFORE_GRACE'] = matrix['PAYMENT_TO_INCOME'] / matrix['LOAN_TO_INCOME'] * 100
    matrix['GRACE_ON'] = 1 * np.array(matrix['PERIOD_ID'] <= matrix['GRACE_PERIOD'])
    
    matrix['RATIO_5'] = 1 * np.array(matrix['LOAN_TO_INCOME'] >= 5)
    matrix['RATIO_10'] = 1 * np.array(matrix['LOAN_TO_INCOME'] >= 10)
    matrix['RATIO_20'] = 1 * np.array(matrix['LOAN_TO_INCOME'] >= 20)
    matrix['RATIO_30'] = 1 * np.array(matrix['LOAN_TO_INCOME'] >= 30)
    matrix['RATIO_40'] = 1 * np.array(matrix['LOAN_TO_INCOME'] >= 40)
    
    matrix['IS_PERIOD_1'] = 1 * np.array(matrix['PERIOD_ID'] == 1)
    matrix['IS_PERIOD_2'] = 1 * np.array(matrix['PERIOD_ID'] == 2)
    matrix['IS_PERIOD_3'] = 1 * np.array(matrix['PERIOD_ID'] == 3)
    
    matrix['IS_CAR_1'] = 1 * np.array(matrix['CAR_CATEGORY'] == 1)
    matrix['IS_CAR_2'] = 1 * np.array(matrix['CAR_CATEGORY'] == 2)
    matrix['IS_CAR_3'] = 1 * np.array(matrix['CAR_CATEGORY'] == 3)
    matrix['IS_CAR_4'] = 1 * np.array(matrix['CAR_CATEGORY'] == 4)
    matrix['IS_CAR_5'] = 1 * np.array(matrix['CAR_CATEGORY'] == 5)
    
    data_table.id_numeric_features.append('BEFORE_GRACE')
    data_table.indicator_features += ['GRACE_ON', 'RATIO_5', 'RATIO_10', 'RATIO_20', 'RATIO_30', 'RATIO_40'
                                     'IS_PERIOD_1', 'IS_PERIOD_2', 'IS_PERIOD_3',
                                     'IS_CAR_1', 'IS_CAR_2', 'IS_CAR_3', 'IS_CAR_4', 'IS_CAR_5']


def _make_is_grace_constant_feature(data_table: DataTable) -> None:
    """ Adds is_grace_constant feature to the DataTable.
    
    Arguments:
        data_table: Structure storing DataFrame with engineered features
            alongside with convenient column names.
    """
    is_grace_constant: List[int] = list()
    matrix: pd.DataFrame = data_table.matrix
        
    key: str; df: pd.DataFrame
    for key, df in matrix.groupby('CONTRACT_ID'):
        df_grace_on: bool = np.array(df['GRACE_ON'])
        check_vals: Union[np.array, List[float]]  = np.array(df['REPAYMENT_ACTUAL'])
        grace_values: Set[float] = set()
        on: int; val: float
        for on, val in zip(df_grace_on, check_vals):
            if on == 1:
                grace_values.add(val)
        append: Union[np.array, List[float]] = np.zeros_like(check_vals)
        if len(grace_values) < 2:
            append += 1
        is_grace_constant += append.tolist()

    matrix['IS_GRACE_CONSTANT'] = np.array(is_grace_constant)
    data_table.indicator_features.append('IS_GRACE_CONSTANT')

    
def _make_target_columns(data_table: DataTable) -> None:
    """ Adds differently transformed target columns to the DataTable.
    
    Arguments:
        data_table: Structure storing DataFrame with engineered features
            alongside with convenient column names.
    """

    matrix: pd.DataFrame = data_table.matrix
    matrix['LOG_REPAYMENT_ACTUAL'] = np.log1p(matrix['REPAYMENT_ACTUAL'])
    matrix['PERCENT_ACTUAL'] = matrix['REPAYMENT_ACTUAL'] / matrix['CONTRACT_SUM'] * 100
    matrix['LOG_PERCENT_ACTUAL'] = np.log1p(matrix['PERCENT_ACTUAL'])

        
def make_source_datatable(data_static: pd.DataFrame, data_repay: pd.DataFrame) -> DataTable:
    """ Performs Feature Engineering and builds DataTable with corresponding information.
    
    Arguments:
        data_static: DataFrame with static information.
        data_repay: DataFrame with information about repayments.
    
    Returns:
        DataTable -- ready to use structure with engineered features.
    """
    data_join: pd.DataFrame = pd.merge(data_static, data_repay, on='CONTRACT_ID')
    data_table: DataTable = DataTable()
    data_join['GENDER'] = 1 * (data_join['GENDER'] == 'M')
    data_table.matrix = data_join
    
    _make_payment_cumulative_and_average_features(data_table)
    _make_payment_history_features(data_table)
    _make_payment_relative_features(data_table)
    _make_payment_log_features(data_table)
    
    _make_regular_log_features(data_table)
    _make_custom_features(data_table)
    _make_is_grace_constant_feature(data_table)    
    _make_target_columns(data_table)
    
    return data_table

## Train-Test Split

In [4]:
def train_test_split(data_table: DataTable) -> Tuple[DataTable, DataTable, DataTable]:
    """ Performs train-test split.
    
    Arguments:
        data_table: Structure storing DataFrame with engineered features
            alongside with convenient column names.
    
    Returns:
        Tuple with three DataTables. The first is training dataset, the second is
        testing dataset, and the third is complete dataset.
    
    """
    contracts: List[str] = np.array(data_table.matrix['CONTRACT_ID'].tolist())
    contracts: Tuple[List[str], List[int]] = np.unique(contracts)
    np.random.shuffle(contracts)
    length: int = contracts.shape[0]
    RATIO: float = 0.1
    test_contracts: List[str]  = contracts[:int(length * RATIO)].tolist()
    train_contracts: List[str] = contracts[int(length * RATIO):].tolist()

    data_learn: pd.DataFrame = copy.deepcopy(data_table.matrix.dropna(axis=0))

    data_train: pd.DataFrame = copy.deepcopy(data_learn[data_learn['CONTRACT_ID'].isin(train_contracts)])
    data_test: pd.DataFrame  = copy.deepcopy(data_learn[data_learn['CONTRACT_ID'].isin(test_contracts)])
    
    train_table: DataTable = copy.deepcopy(data_table)
    train_table.matrix = data_train
    test_table: DataTable = copy.deepcopy(data_table)
    test_table.matrix = data_test
    dropna_table: DataTable = copy.deepcopy(data_table)
    dropna_table.matrix = data_learn
    
    return train_table, test_table, dropna_table
    

## Logic calling defined API above and saving the result.

In [5]:
all_data_table: DataTable = make_source_datatable(data_static, data_repay)
train_data_table: DataTable; test_data_table: DataTable; dropna_table: DataTable
train_data_table, test_data_table, dropna_table = train_test_split(all_data_table)

if not os.path.exists('prepared_data'):
    os.makedirs('prepared_data')

def check_existence(list_of_paths: List[str]) -> bool:
    """ Checks if all of the paths exist in operating system. 
    
    Arguments:
        list_of_paths -- List containing paths to check.
        
    Returns:
        True if all of the paths are valid, and False otherwise.
    """
    item: bool
    for item in list_of_paths:
        if not os.path.exists(item):
            return False
    return True

paths_list: List[str] = ['prepared_data/train_data.csv', 'prepared_data/train_data.jbl', 
              'prepared_data/test_data.csv', 'prepared_data/test_data.jbl',
              'prepared_data/all_data_dropped_na.csv', 'prepared_data/all_data_dropped_na.jbl',
              'prepared_data/all_data_with_na.csv', 'prepared_data/all_data_with_na.jbl'
             ]

if not check_existence(paths_list):
    train_data_table.matrix.to_csv('prepared_data/train_series.csv')
    joblib.dump(train_data_table,  "prepared_data/train_data.jbl")
    test_data_table.matrix.to_csv('prepared_data/test_series.csv')
    joblib.dump(test_data_table,  "prepared_data/test_data.jbl")
    all_data_table.matrix.to_csv('prepared_data/all_data_with_na.csv')
    joblib.dump(all_data_table,  "prepared_data/all_data_with_na.jbl")
    dropna_table.matrix.to_csv('prepared_data/all_data_dropped_na.csv')
    joblib.dump(dropna_table,  'prepared_data/all_data_dropped_na.jbl')